# Bot de Telegram para consultas de regulación ambiental

In [1]:
import import_ipynb
from config import * #Archivo donde se encuentra el token, no se encuentra pública para evitar conflicto tipo request
import telebot
import threading
from datetime import datetime
import locale

from MakeSens import MakeSens
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

importing Jupyter notebook from config.ipynb


/home/hogar/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
#IDE de cada detector situado en cuatro localizaciones de Bucaramanga
est = {'Colegio Tecnológico':'mE1_00004','Colegio Santander':'mE1_00005',
       'Colegio la Normal':'mE1_00007','UIS: Grupo Halley':'E2_00023'}

In [3]:
def ShowImg(IDE, start,end,freq, place, opt): #IDE de minieva, tiempo iniial, tiempo final, frecuencia, lugar, opción a visualizar
    data = MakeSens.download_data(IDE,start,end,freq)
    data.index = pd.to_datetime(data.index)
    plt.figure(figsize = (15,10))
    
    if opt == 'Humedad ambiental':
        plt.plot(data.index.strftime("%H:%M"), data['humedad'], label=r'Medición Principal')
        plt.plot(data.index.strftime("%H:%M"), data['humedad2'], label=r'Medición Auxiliar')

        plt.ylabel('Humedad ambiental relativa en {} '.format(place) + r'$[\%RH]$', fontsize=20)
        plt.xlabel(r'Tiempo [H]', fontsize=20)

        plt.legend(fontsize=20)
        plt.savefig('./{}_{}.png'.format(ans,site))
    
    elif opt =='Material Particulado':
        plt.plot(data.index.strftime("%H:%M"), data['pm1_1'], label=r'0.3 - 1.0 $\mu m$')
        plt.plot(data.index.strftime("%H:%M"), data['pm25_1'], label=r'1.0 - 2.5 $\mu m$')
        plt.plot(data.index.strftime("%H:%M"), data['pm10_1'], label=r'2.5 - 10 $\mu m$')

        plt.ylabel('Cantidad de material partículado en {} '.format(place) + r'$[\dfrac{\mu g}{m^3}]$', fontsize=20)
        plt.xlabel(r'Tiempo [H]', fontsize=20)

        plt.legend(fontsize=20)
        plt.savefig('./{}_{}.png'.format(ans,site))
        
    elif opt=='Temperatura':
        plt.plot(data.index.strftime("%H:%M"), data['temperatura'], label=r'Medición Principal')
        plt.plot(data.index.strftime("%H:%M"), data['temperatura2'], label=r'Medición Auxiliar')

        plt.ylabel('Temperatura en {} '.format(place) + r'$[^{\circ}C]$', fontsize=20)
        plt.xlabel(r'Tiempo [H]', fontsize=20)

        plt.legend(fontsize=20)
        plt.savefig('./{}_{}.png'.format(ans,site))
        

In [4]:
bot = telebot.TeleBot(TELEGRAM_TOKEN) #Instanciamos objeto bot

In [5]:
ans = "" #Variable de parámetro de consulta
start_time="" #Variable de tiempo inicial
end_time="" #Variable de tiempo final
site="" #Sitio donde se sacan los datos (exa-minieva)

@bot.message_handler(commands=['start']) #Se ejecuta con el comando /start del usuario

def cmd_start(message):
    #username = message.from_user.username
    if message.from_user.username:
        username = message.from_user.username
        
    elif message.from_user.first_name and message.from_user.last_name:
        username = f"{message.from_user.first_name} {message.from_user.last_name}"
        
    else:
        username = f"{message.from_user.first_name}"
        
    
    text = "'Hola {}, bienvenido a BotAir, en el cual podrás efectuar las consultas que desees acerca de la calidad del aire en Bucaramanga', *¿Qué parámetros deseas consultar: ?*".format(username) + "\n"
    text+="Humedad ambiental" + "\n"
    text+="Material Particulado" +"\n"
    text+="Temperatura" +"\n"
    bot.reply_to(message, text, parse_mode='MarkdownV2')
    bot.register_next_step_handler(message, answers)
    
def answers(message):
    global ans
    options= ['Humedad ambiental','Material Particulado','Temperatura']
    ans = message.text
    for i in range(len(options)):
        if ans != options[i]:
            continue
        elif ans == options[i]:
            bot.send_message(message.chat.id, "Has seleccionado {}, ¿Desde qué fecha deseas iniciar?\n  Escribela de la forma AAAA-MM-DD hh:mm:ss".format(options[i]))
            break
        else:
            bot.send_message(message.chat.id, "Opción no encontrada".format(options[i]))

    bot.register_next_step_handler(message, quest1)

def quest1(message):
    global start_time
    start_time = message.text
    bot.send_message(message.chat.id, "¿Hasta qué fecha deseas consultar?\n  Escribela de la forma AAAA-MM-DD hh:mm:ss")
    bot.register_next_step_handler(message, quest2)
    
def quest2(message):
    global end_time
    end_time = message.text
    text = "*¿En donde deseas realizar la consulta?*"+"\n"
    text+="Colegio Tecnológico" + "\n"
    text+="Colegio Santander" +"\n"
    text+="Colegio la Normal" +"\n"
    text+="UIS: Grupo Halley" +"\n"
    bot.send_message(message.chat.id, text, parse_mode='MarkdownV2')
    bot.register_next_step_handler(message, where)
    
def where(message):
    global site
    site = message.text
    try:
        ShowImg(est[site],start_time,end_time,'H',site, ans)
        img = open('./{}_{}.png'.format(ans,site),'rb')
        bot.send_photo(message.chat.id, img,'{} medido en {} entre {} y {}'.format(ans,site,start_time,end_time))
        os.remove('./{}_{}.png'.format(ans,site))
    except:
        bot.send_message(message.chat.id, 'Ocurrió un error, ¿Podrías intentar nuevamente?')

In [6]:
def getting():
    bot.infinity_polling() #Revisión de bot mientras se está en ejecución

if __name__=='__main__':
    print('Iniciando')
    bot_thread = threading.Thread(name='bot_thread', target=getting) #Adjudicación de hilo para revisión de bot
    bot_thread.start()
    print('Fin')

Iniciando
Fin
